In [7]:
import numpy as np
import pandas as pd
import csv

import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import *
from collections import Counter
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
import torch

## KLUE + TF-IDF

### KLUE

In [184]:
# 파일 가져오기
df_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/최종_데이터셋7(월,일,색상,RGB).csv"

df = pd.read_csv(df_file) #, encoding='CP949'
df.drop(["일자", "선택한 색상", "중앙 색상들"], axis=1, inplace=True)

In [218]:
# user_input = "졸업에 어울리는 꽃을 추천해줘"
user_input = "여자친구랑 곧 6주년이야"

In [219]:
# NLP 처리 할 데이터만 가져오기
df_nlp = df[["꽃", "꽃말", "설명", "색상"]]
print(df_nlp.shape)
df_nlp.head(5)

# 꽃말, 설명 합치기 (색상은 유지)
df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['설명']

# 특수문자 및 띄어쓰기 제거 함수
import re

def remove_special_characters(text):
    if not isinstance(text, str):
        text = str(text)

    # 특수문자 제거
    pattern = r'[^\w\s\.]' # 문자, 공백문자, 마침표 제외 제거
    clean_text = re.sub(pattern, '', text)

    # 한자 제거
    pattern = r'[\u4e00-\u9fff]' # 중국어 한자의 유니코드 시작과 끝 제거
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = ' '.join(clean_text.split())
    return clean_text

df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
df_nlp['색상'] = df_nlp['색상'].apply(remove_special_characters)

(837, 4)


C:\Users\pc\AppData\Local\Temp\ipykernel_9456\2385004683.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['설명']
C:\Users\pc\AppData\Local\Temp\ipykernel_9456\2385004683.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
C:\Users\pc\AppData\Local\Temp\ipykernel_9456\2385004683.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [220]:
# Klue-RoBERTa 모델과 토크나이저 불러오기
model_name = 'klue/roberta-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# [CLS]와 [SEP] 토큰을 추가하는 함수
def add_special_tokens(text):
    # 문장을 마침표를 기준으로 분리
    sentences = text.split('. ')
    # 각 문장에 [CLS]와 [SEP] 토큰 추가
    sentences_with_special_tokens = ['[CLS] ' + sentence + ' [SEP]' for sentence in sentences]
    # 다시 문장으로 결합
    text_with_special_tokens = ' '.join(sentences_with_special_tokens)
    return text_with_special_tokens

df_nlp['설명2'] = df_nlp['설명'].apply(add_special_tokens)
df_nlp['색상2'] = df_nlp['색상'].apply(add_special_tokens)
df_nlp.head(10)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\pc\AppData\Local\Temp\ipykernel_9456\213206039.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명2'] = df_nlp['설명'].apply(add_special_tokens)
C:\Users\pc\AppData\Local\Temp\ipykernel_9456\213206039.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,꽃,꽃말,설명,색상,설명2,색상2
0,각시붓꽃,"부끄러움, 세련됨",부끄러움 세련됨. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 ...,보라,[CLS] 부끄러움 세련됨 [SEP] [CLS] 각시라 하면 이제 막 시집 온 새색...,[CLS] 보라 [SEP]
1,감국,그윽한 향기,그윽한 향기. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인...,노랑,[CLS] 그윽한 향기 [SEP] [CLS] 가을 산야는 국화과 식물들 차지다 [S...,[CLS] 노랑 [SEP]
2,개나리,"희망,깊은 정, 달성",희망깊은 정 달성. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃 개...,노랑,[CLS] 희망깊은 정 달성 [SEP] [CLS] 우리나라 전역에서 봄 소식을 가장...,[CLS] 노랑 [SEP]
3,개나리,희망,희망. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리 개나리...,노랑,[CLS] 희망 [SEP] [CLS] 개나리 봄을 알리는 전령사 하면 가장 먼저 떠...,[CLS] 노랑 [SEP]
4,갯개미취,추억,추억. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 ...,보라,[CLS] 추억 [SEP] [CLS] 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가...,[CLS] 보라 [SEP]
5,거베라,"신비, 수수께끼",신비 수수께끼. 거베라만큼 크고 화려한 꽃도 드물다. 부를 상징하기도 한다. 거베라...,분홍,[CLS] 신비 수수께끼 [SEP] [CLS] 거베라만큼 크고 화려한 꽃도 드물다 ...,[CLS] 분홍 [SEP]
6,검은포플라,용기,용기. 당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다. 그렇다고 해서 ...,빨강,[CLS] 용기 [SEP] [CLS] 당신은 용기가 있어 주위 사람들도 당신을 의지...,[CLS] 빨강 [SEP]
7,겹 공작초,첫눈에 반함,첫눈에 반함. 자기가 가장 아름답다고 믿는 세 여신이 있었다. 제우스의 아내 헤라 ...,보라,[CLS] 첫눈에 반함 [SEP] [CLS] 자기가 가장 아름답다고 믿는 세 여신이...,[CLS] 보라 [SEP]
8,겹 과꽃,나의 사랑은 당신보다깊다,나의 사랑은 당신보다깊다. 괴테의 희곡 파우스트 에 등장하는 소녀 마가렛이 꽃잎을 ...,보라,[CLS] 나의 사랑은 당신보다깊다 [SEP] [CLS] 괴테의 희곡 파우스트 에 ...,[CLS] 보라 [SEP]
9,겹 소국,청초,청초. 국화 품종 중 작은 화형을 가진 국화를 소국이라고 하며 스프레이 형태를 띄고...,분홍,[CLS] 청초 [SEP] [CLS] 국화 품종 중 작은 화형을 가진 국화를 소국이...,[CLS] 분홍 [SEP]


In [221]:
# 문장을 벡터화 변환하는 함수
def get_sentence_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True) # 최대 길이 초과 시 잘라내기, 작은 경우 패딩 진행
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy() # 각 토큰 벡터의 첫 번째 벡터 확인 (CLS 토큰 벡터)

# 설명 벡터화
df_nlp['설명_벡터'] = df_nlp['설명2'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist()) # 데이터마다 (1, 768) 차원 벡터 생성
df_nlp['색상_벡터'] = df_nlp['색상2'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist())

In [222]:
df_nlp['코사인유사도'] = df_nlp['설명_벡터'].apply(lambda x: cosine_similarity(user_vetor1, x)[0, 0])
top3 = df_nlp.nlargest(3, '코사인유사도')
top3

,꽃,꽃말,설명,색상,설명2,색상2,설명_벡터,색상_벡터,코사인유사도
718,튤립,아름다운 눈동자,아름다운 눈동자. 들에 핀 꽃처럼 아름다운 당신. 사랑하는 사람을 실망시키지 않도록...,빨강,[CLS] 아름다운 눈동자 [SEP] [CLS] 들에 핀 꽃처럼 아름다운 당신 [S...,[CLS] 빨강 [SEP],"[[0.20491516590118408, 0.134373739361763, 0.08...","[[0.1326621174812317, 0.29782605171203613, -0....",0.931099
420,수선화,신비,신비. 당신은 상처받는 것이 두려워 뒤로 물러서는 타입입니다. 이제는 용기가 필요한...,흰색,[CLS] 신비 [SEP] [CLS] 당신은 상처받는 것이 두려워 뒤로 물러서는 타...,[CLS] 흰색 [SEP],"[[0.20341157913208008, 0.17813554406166077, 0....","[[0.11117629706859589, 0.28557634353637695, -0...",0.930849
421,수선화,신비,신비. 당신은 상처받는 것이 두려워 뒤로 물러서는 타입입니다. 이제는 용기가 필요한...,흰색,[CLS] 신비 [SEP] [CLS] 당신은 상처받는 것이 두려워 뒤로 물러서는 타...,[CLS] 흰색 [SEP],"[[0.20341157913208008, 0.17813554406166077, 0....","[[0.11117629706859589, 0.28557634353637695, -0...",0.930849


'졸업' 키워드 잘 잡는지 살펴보기 (user_input: 위에 넣었을 때 결과값임)
- '졸업'이 있는 프리지아, 사스레피 모두 유사도가 꽤 괜찮게 나옴

In [189]:
# df_nlp[df_nlp['꽃']=='프리지아']

,꽃,꽃말,설명,색상,설명2,색상2,설명_벡터,색상_벡터,코사인유사도
763,프리지아,"순결, 순진한 마음",순결 순진한 마음. 꽃집에 프리지아가 나오기 시작하면 진한 향기와 함께 비로소 계절...,노랑,[CLS] 순결 순진한 마음 [SEP] [CLS] 꽃집에 프리지아가 나오기 시작하면...,[CLS] 노랑 [SEP],"[[-0.014409363269805908, 0.18728667497634888, ...","[[0.11612437665462494, 0.3139854669570923, -0....",0.894718
764,프리지아,새로운 시작을 응원합니다,새로운 시작을 응원합니다. 취업 이직. 특유의 진한 향기를 가진 프리지아는 시작을 ...,노랑,[CLS] 새로운 시작을 응원합니다 [SEP] [CLS] 취업 이직 [SEP] [C...,[CLS] 노랑 [SEP],"[[0.11334358900785446, 0.22307468950748444, 0....","[[0.11612437665462494, 0.3139854669570923, -0....",0.923509
765,프리지아,순결,순결. 순진함 순정 청순함을 그림으로 그려 놓은듯한 사람. 달콤한 거짓말에 그만 속...,노랑,[CLS] 순결 [SEP] [CLS] 순진함 순정 청순함을 그림으로 그려 놓은듯한 ...,[CLS] 노랑 [SEP],"[[0.28162088990211487, 0.12160040438175201, -0...","[[0.11612437665462494, 0.3139854669570923, -0....",0.922263


In [190]:
# df_nlp[df_nlp['꽃']=='사스레피나무꽃']

,꽃,꽃말,설명,색상,설명2,색상2,설명_벡터,색상_벡터,코사인유사도
366,사스레피나무꽃,당신은 소중합니다.,당신은 소중합니다.. 오래 전 졸업식이나 입학식의 꽃다발 장식에 빠지지 않고 쓰였던...,흰색,[CLS] 당신은 소중합니다. [SEP] [CLS] 오래 전 졸업식이나 입학식의 꽃...,[CLS] 흰색 [SEP],"[[0.0868799239397049, 0.2841452956199646, 0.26...","[[0.11117629706859589, 0.28557634353637695, -0...",0.916003


### tf-idf

In [191]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [223]:
# TfidfVectorizer 객체 생성
tfidf_vectorizer = TfidfVectorizer()

# 문서 집합을 tfidf 벡터화
tfidf_matrix = tfidf_vectorizer.fit_transform(df_nlp['설명'])

In [224]:
# 사용자 입력 TF-IDF 벡터화
user_tfidf = tfidf_vectorizer.transform([user_input])

In [227]:
# 각 문서와 사용자 입력 간의 코사인 유사도 계산
similarities = cosine_similarity(user_tfidf, tfidf_matrix)
similarities = pd.DataFrame(similarities.reshape(-1,1))
df_nlp['tfidf_Similarity'] = similarities

In [233]:
top3 = df_nlp.nlargest(3, 'tfidf_Similarity').drop_duplicates(subset='꽃')
top3

,꽃,꽃말,설명,색상,설명2,색상2,설명_벡터,색상_벡터,코사인유사도,tfidf_Similarity,combined_Similarity
0,각시붓꽃,"부끄러움, 세련됨",부끄러움 세련됨. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 ...,보라,[CLS] 부끄러움 세련됨 [SEP] [CLS] 각시라 하면 이제 막 시집 온 새색...,[CLS] 보라 [SEP],"[[0.07368195801973343, 0.15451766550540924, 0....","[[0.15296272933483124, 0.3076050877571106, -0....",0.898629,0.0,NaN
1,감국,그윽한 향기,그윽한 향기. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인...,노랑,[CLS] 그윽한 향기 [SEP] [CLS] 가을 산야는 국화과 식물들 차지다 [S...,[CLS] 노랑 [SEP],"[[0.08965592086315155, 0.2825712561607361, 0.1...","[[0.11612437665462494, 0.3139854669570923, -0....",0.895026,0.0,NaN
2,개나리,"희망,깊은 정, 달성",희망깊은 정 달성. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃 개...,노랑,[CLS] 희망깊은 정 달성 [SEP] [CLS] 우리나라 전역에서 봄 소식을 가장...,[CLS] 노랑 [SEP],"[[0.13718879222869873, 0.23945458233356476, 0....","[[0.11612437665462494, 0.3139854669570923, -0....",0.908020,0.0,NaN


얘는 모두 tfidf가 0인 듯? ㅠ

In [211]:
# #"졸업에 어울리는 꽃을 추천해줘"일 때
# top3 = df_nlp.nlargest(3, 'tfidf_Similarity').drop_duplicates(subset='꽃')
# top3

,꽃,꽃말,설명,색상,설명2,색상2,설명_벡터,색상_벡터,코사인유사도,tfidf_Similarity
755,풀몬티 장미,사랑의 맹세,사랑의 맹세. 성년의 날 발렌타인데이 로즈데이. 풀몬티 full monty 는 벌거...,빨강,[CLS] 사랑의 맹세 [SEP] [CLS] 성년의 날 발렌타인데이 로즈데이 [SE...,[CLS] 빨강 [SEP],"[[0.353645384311676, 0.1402251422405243, 0.206...","[[0.1326621174812317, 0.29782605171203613, -0....",0.910794,0.137566
105,노랑 히야신,승부,승부. 당신은 언제나 엄격하지 않으면 못견뎌 하는 성격입니다. 엄격히 하는 것도 시...,노랑,[CLS] 승부 [SEP] [CLS] 당신은 언제나 엄격하지 않으면 못견뎌 하는 성...,[CLS] 노랑 [SEP],"[[0.28819435834884644, 0.22517266869544983, 0....","[[0.11612437665462494, 0.3139854669570923, -0....",0.921608,0.133510
383,살비아,"불타는 마음, 정열",불타는 마음 정열. 우리말로는 깨꽃이라 불렸던 살비아 여름부터 초가을까지 핀다. 꽃...,빨강,[CLS] 불타는 마음 정열 [SEP] [CLS] 우리말로는 깨꽃이라 불렸던 살비아...,[CLS] 빨강 [SEP],"[[0.1124977320432663, 0.31079885363578796, 0.1...","[[0.1326621174812317, 0.29782605171203613, -0....",0.908047,0.133089


해당 경우, 그냥 명사로 '졸업'이라고 하면 가장 위에 프리지아가 나옴 !
- tf-idf 사용할 때는 문장으로 만들지 않는 게 나음

### 유사도 합치기

In [230]:
# 정규화 (0-1 범위로)
semantic_similarities = (df_nlp['코사인유사도'] - df_nlp['코사인유사도'].min()) / (df_nlp['코사인유사도'].max() - df_nlp['코사인유사도'].min())
tfidf_similarities = (df_nlp['tfidf_Similarity'] - df_nlp['tfidf_Similarity'].min()) / (df_nlp['tfidf_Similarity'].max() - df_nlp['tfidf_Similarity'].min())

In [231]:
# 가중치 적용 및 결합
semantic_weight=0.7
keyword_weight = 1 - semantic_weight

combined_scores = (semantic_weight * semantic_similarities + 
                    keyword_weight * tfidf_similarities)

In [217]:
df_nlp['combined_Similarity'] = combined_scores
top3 = df_nlp.nlargest(3, 'combined_Similarity').drop_duplicates(subset='꽃')
top3

,꽃,꽃말,설명,색상,설명2,색상2,설명_벡터,색상_벡터,코사인유사도,tfidf_Similarity,combined_Similarity
105,노랑 히야신,승부,승부. 당신은 언제나 엄격하지 않으면 못견뎌 하는 성격입니다. 엄격히 하는 것도 시...,노랑,[CLS] 승부 [SEP] [CLS] 당신은 언제나 엄격하지 않으면 못견뎌 하는 성...,[CLS] 노랑 [SEP],"[[0.28819435834884644, 0.22517266869544983, 0....","[[0.11612437665462494, 0.3139854669570923, -0....",0.921608,0.133510,0.878192
728,팔로마 장미,고백 성공,고백 성공. 성년의 날 발렌타인데이 로즈데이. 레드핑크 그라데이션 색감이 강렬해서 ...,빨강,[CLS] 고백 성공 [SEP] [CLS] 성년의 날 발렌타인데이 로즈데이 [SEP...,[CLS] 빨강 [SEP],"[[0.27577686309814453, 0.12073814868927002, 0....","[[0.1326621174812317, 0.29782605171203613, -0....",0.920509,0.124896,0.846325
403,설앵초,비할바없는 아름다움,비할바없는 아름다움. 혼자서 무슨 일이든 해내고 말기 때문에 다른 사람앞에서 장점을...,보라,[CLS] 비할바없는 아름다움 [SEP] [CLS] 혼자서 무슨 일이든 해내고 말기...,[CLS] 보라 [SEP],"[[0.1718728244304657, 0.14881718158721924, 0.1...","[[0.15296272933483124, 0.3076050877571106, -0....",0.929617,0.070108,0.835247
